In [1]:
from pt.data.dataset_mapper import DatasetMapperTwoCropSeparate
from pt import add_config
import pt.data.datasets.builtin
from pt.data.build import build_detection_targetonly_loader_two_crops

from pt.engine.FLtrainer import FLtrainer
from pt.engine.trainer import PTrainer
from pt.modeling.meta_arch.multi_teacher import MultiTSModel
from pt.modeling.meta_arch.rcnn import GuassianGeneralizedRCNN
from pt.modeling.backbone.vgg import build_vgg_backbone
from pt.modeling.proposal_generator.rpn import GuassianRPN
from pt.modeling.roi_heads.roi_heads import GuassianROIHead
from pt.checkpoint.detection_checkpoint import DetectionTSCheckpointer
from pt.modeling.meta_arch.ts_ensemble import EnsembleTSModel



from detectron2.config import get_cfg
from detectron2.data import build_detection_test_loader
from detectron2.checkpoint import DetectionCheckpointer

import os


from Nb_utils import *

NOTE! Installing ujson may make loading annotations faster.


/home/superorange5/.local/lib/python3.8/site-packages/torchvision/transforms/transforms.py:803: UserWarning: Argument interpolation should be of type InterpolationMode instead of int. Please, use InterpolationMode enum.
  warnings.warn(


# A. check datamapper

In [4]:
dataset_name = 'VOC2007_bddvalsmall'
cfg = setup("configs/pt/final_c2b.yaml")
mapper = DatasetMapperTwoCropSeparate(cfg, True)
test_data_loader = build_detection_test_loader(cfg, dataset_name,mapper)


In [5]:
data_loader_iter = iter(test_data_loader)
test_data = data_loader_iter.next()

In [ ]:
test_data[0]

# B. check  build_detection_targetonly_loader_two_crops

In [7]:
dataset_name = 'VOC2007_bddvalsmall'
cfg = setup("configs/pt/final_c2b.yaml")
mapper = DatasetMapperTwoCropSeparate(cfg, True)
test_data_loader= build_detection_targetonly_loader_two_crops(cfg, mapper)

In [10]:
data_loader_iter = iter(test_data_loader)

In [13]:
data = next(data_loader_iter)

In [ ]:
data[1]

# C. build model, load weight

In [2]:
cfg_path = "configs/multi-teacher/c2b_1cls.yaml"
cfg = setup(cfg_path)

                              
#DetectionCheckpointer(multi_teacher_model).resume_or_load(student_model_path, resume=False)


## C.1 build model structure

### C.1.a get specific model path

In [4]:
student_model_path = 'keep_experiments/FedPT_ck2b/avg_ck2b/pt_ck2b_AVG_round3.pth'

Trainer=PTrainer
model = Trainer.build_model(cfg)
model_teacher = Trainer.build_model(cfg)
ensem_ts_model = EnsembleTSModel(model_teacher, model)    

DetectionCheckpointer(ensem_ts_model).resume_or_load(student_model_path, resume=False)


-------- pretrained model loaded ---------
-------- pretrained model loaded ---------


{'matching_heuristics': True}

### C.1.b get model path from cfg

In [3]:
model_path_list = cfg.MODEL.TEACHER_PATH
student_model_path = cfg.MODEL.STUDENT_PATH

In [4]:

# model_folder = 'keep_experiments'

# model_k2b_path = os.path.join(model_folder,'k2b','model_final.pth')
# model_c2b_path = os.path.join(model_folder,'c2b','model_{0:07d}.pth'.format(11999))
# student_model_path = 'keep_experiments/FedPT_ck2b/avg_ck2b/pt_ck2b_AVG_round3.pth'

# model_path_list = [model_k2b_path,model_c2b_path]


# build student model
Trainer =FLtrainer
student_model = Trainer.build_model(cfg)

# build teacher model
model_teacher_list = []                              
for idx in range(len(model_path_list)):
    teacher_model = Trainer.build_model(cfg)
    model_teacher_list.append(teacher_model)
multi_teacher_model = MultiTSModel(model_teacher_list, student_model)    

-------- pretrained model loaded ---------
-------- pretrained model loaded ---------
-------- pretrained model loaded ---------


## C.2 load weight

In [5]:
model_list=[]
for model_teacher_path in model_path_list:
    print("load teacher model:{} ".format(model_teacher_path))
    model_with_weight = load_TSmodel(cfg_path, model_teacher_path)
    model_list.append(model_with_weight)
    
print("load student model:{} ".format(student_model_path))
student_initial_backbone = load_TSmodel(cfg_path, student_model_path)

load teacher model:keep_experiments/k2b/model_final.pth 
-------- pretrained model loaded ---------
-------- pretrained model loaded ---------
load teacher model:keep_experiments/c2b/model_0011999.pth 
-------- pretrained model loaded ---------
-------- pretrained model loaded ---------
load student model:keep_experiments/FedPT_ck2b/avg_ck2b/pt_ck2b_AVG_round3.pth 
-------- pretrained model loaded ---------
-------- pretrained model loaded ---------


## C.3 feed model weight to multi-teacher model structure

In [6]:
from collections import OrderedDict

#-----load multi teacher
for i, model in enumerate(model_list):
    new_teacher_dict = OrderedDict()
    
    source_model_dict = model.modelStudent.state_dict()
    
    for key, value in source_model_dict.items():
        if key in multi_teacher_model.modelTeacher[i].state_dict().keys():
            new_teacher_dict[key] = value    
    multi_teacher_model.modelTeacher[i].load_state_dict(new_teacher_dict)
    
#----------load student
new_student_dict = OrderedDict()
pseudo_model_dict = student_initial_backbone.modelStudent.state_dict()
for key, value in pseudo_model_dict.items():    
    if key in multi_teacher_model.modelStudent.state_dict().keys():
        new_student_dict[key] = value
multi_teacher_model.modelStudent.load_state_dict(new_student_dict)    

<All keys matched successfully>

## C.4 check weight

In [ ]:
multi_teacher_model.modelTeacher[0].state_dict()['backbone.vgg_block4.0.conv1.weight']

# D. initial trainer (include build model and load weight)

In [31]:
cfg_path = "configs/pt/final_c2b_1class.yaml"
cfg = setup(cfg_path)
Trainer2 =PTrainer(cfg)


-------- pretrained model loaded ---------
-------- pretrained model loaded ---------
-------- pretrained model loaded ---------


WarmupMultiStepLR is deprecated! Use LRMultipilier with fvcore ParamScheduler instead!


In [33]:
data = next(Trainer2._trainer._data_loader_iter)
        # data_q and data_k from different augmentations (q:strong, k:weak)
        # label_strong, label_weak, unlabed_strong, unlabled_weak
label_data_q, label_data_k, unlabel_data_q, unlabel_data_k = data

In [35]:
pesudo_proposals_roih_unsup_k, _ = Trainer2.process_pseudo_label(
                proposals_roih_unsup_k, "roih", "all"
            )

In [48]:
unlabel_data_q = Trainer2.remove_label(unlabel_data_q)

unlabel_data_q = Trainer2.add_label(
                unlabel_data_q, pesudo_proposals_roih_unsup_k
            )

In [60]:
has_pseudo_boxes = unlabel_data_q[0]['instances'].has('pseudo_boxes')

In [ ]:
unlabel_data_q

In [61]:
has_pseudo_boxes

True

In [53]:
record_all_unlabel_data, _, _, _ = Trainer2.model(
                unlabel_data_q, branch="unsupervised", danchor=True
            )

AssertionError: get_event_storage() has to be called inside a 'with EventStorage(...)' context!

In [3]:
Trainer =FLtrainer(cfg)


-------- pretrained model loaded ---------
-------- pretrained model loaded ---------
-------- pretrained model loaded ---------


WarmupMultiStepLR is deprecated! Use LRMultipilier with fvcore ParamScheduler instead!


load teacher model:keep_experiments/k2b/model_final.pth 
-------- pretrained model loaded ---------
-------- pretrained model loaded ---------
load teacher model:keep_experiments/c2b/model_0011999.pth 
-------- pretrained model loaded ---------
-------- pretrained model loaded ---------
load student model:keep_experiments/FedPT_ck2b/avg_ck2b/pt_ck2b_AVG_round3.pth 
-------- pretrained model loaded ---------
-------- pretrained model loaded ---------


## D.1 check weight

### D.1.a check student model weight

In [15]:
Trainer.model.state_dict()['backbone.vgg_block4.0.conv1.weight'][0]

tensor([[[-0.0137, -0.0116, -0.0033],
         [-0.0155, -0.0198, -0.0163],
         [-0.0222, -0.0169, -0.0093]],

        [[ 0.0027,  0.0018,  0.0166],
         [-0.0079, -0.0036,  0.0065],
         [ 0.0034, -0.0087, -0.0048]],

        [[ 0.0173, -0.0144, -0.0179],
         [ 0.0332,  0.0345, -0.0020],
         [-0.0202,  0.0205,  0.0155]],

        ...,

        [[-0.0042, -0.0144,  0.0045],
         [-0.0033, -0.0228, -0.0189],
         [-0.0039,  0.0025,  0.0079]],

        [[-0.0238, -0.0171, -0.0193],
         [-0.0165, -0.0141,  0.0041],
         [ 0.0024,  0.0097,  0.0248]],

        [[ 0.0025, -0.0053, -0.0080],
         [-0.0036, -0.0036, -0.0048],
         [-0.0152, -0.0060, -0.0088]]], device='cuda:0')

### D.1.b check teacher model weight

In [16]:
Trainer.model_teacher_list[0].state_dict()['backbone.vgg_block4.0.conv1.weight'][0]

tensor([[[-1.3013e-02, -1.7674e-02, -8.7242e-03],
         [-6.6417e-03, -2.8179e-02, -1.7347e-02],
         [-1.4393e-02, -2.5782e-02, -1.2142e-02]],

        [[-7.6653e-03,  7.4842e-03,  2.7621e-02],
         [-2.5077e-03,  9.7199e-03,  3.3551e-02],
         [-1.8230e-02, -2.0012e-03,  1.7729e-02]],

        [[-1.1370e-02, -1.1831e-02, -1.1899e-02],
         [-7.7858e-04,  6.0030e-02,  2.9155e-02],
         [-2.8562e-02,  2.3417e-02,  3.1388e-02]],

        ...,

        [[-3.1497e-03, -1.9339e-03, -4.5325e-05],
         [ 1.7527e-04, -8.6480e-03, -2.3997e-02],
         [ 3.6784e-03,  2.5598e-02,  1.0689e-02]],

        [[-4.1432e-02, -3.3276e-02, -2.5533e-02],
         [-1.2606e-02, -1.8507e-02, -1.0148e-02],
         [-3.0346e-02, -7.8686e-03,  1.8107e-02]],

        [[ 1.1084e-02, -2.5984e-03,  2.4725e-03],
         [-1.0399e-02, -8.0159e-03, -3.6350e-03],
         [-1.2528e-02, -2.8805e-03, -2.7278e-03]]], device='cuda:0')

## D.2 load data

In [56]:
unlabel_data_q, unlabel_data_k  = next(Trainer._trainer._data_loader_iter)


In [6]:
unlabel_data_q[0]

{'file_name': 'data/VOC2007_bddval/JPEGImages/b1c66a42-6f7d68ca.jpg',
 'image_id': 'b1c66a42-6f7d68ca',
 'height': 720,
 'width': 1280,
 'instances': Instances(num_instances=19, image_height=600, image_width=1067, fields=[gt_boxes: Boxes(tensor([[ 171.7203,  281.6667,  235.9070,  324.1667],
         [  39.1789,  286.6667,  109.2008,  335.0000],
         [ 205.8977,  286.6667,  290.0906,  330.8333],
         [   0.0000,  280.8333,   44.1805,  336.6667],
         [ 570.1781,  297.5000,  601.8547,  328.3333],
         [ 588.5172,  303.3333,  612.6914,  326.6667],
         [ 606.0226,  305.0000,  635.1984,  335.0000],
         [ 624.3617,  301.6667,  675.2109,  341.6667],
         [ 656.0383,  298.3333,  756.0695,  355.0000],
         [ 733.5625,  313.3333,  801.0836,  377.5000],
         [ 780.2438,  280.0000, 1011.1492,  404.1667],
         [1003.6469,  346.6667, 1067.0000,  442.5000],
         [ 460.1437,  296.6667,  471.8141,  307.5000],
         [ 478.4828,  293.3333,  489.3195,  308.

In [57]:
unlabel_data_q = Trainer.remove_label(unlabel_data_q)

In [9]:
unlabel_data_q[0]

{'file_name': 'data/VOC2007_bddval/JPEGImages/b1c66a42-6f7d68ca.jpg',
 'image_id': 'b1c66a42-6f7d68ca',
 'height': 720,
 'width': 1280,
 'image': tensor([[[254, 255, 255,  ...,  57,  54,  51],
          [243, 250, 253,  ...,  56,  53,  50],
          [206, 231, 243,  ...,  54,  51,  48],
          ...,
          [ 49,  49,  49,  ...,  28,  28,  28],
          [ 47,  47,  47,  ...,  26,  26,  26],
          [ 47,  47,  47,  ...,  25,  25,  25]],
 
         [[254, 255, 254,  ...,  89,  86,  83],
          [241, 249, 251,  ...,  88,  85,  82],
          [202, 227, 239,  ...,  86,  83,  80],
          ...,
          [ 58,  58,  58,  ...,  23,  23,  23],
          [ 56,  56,  56,  ...,  21,  21,  21],
          [ 56,  56,  56,  ...,  20,  20,  20]],
 
         [[222, 219, 216,  ..., 124, 121, 118],
          [207, 215, 217,  ..., 123, 120, 117],
          [174, 199, 211,  ..., 121, 118, 115],
          ...,
          [ 61,  61,  61,  ...,  20,  20,  20],
          [ 59,  59,  59,  ...,  18,

In [58]:
 unlabel_data_q = Trainer.add_label( unlabel_data_q, list_instances)

In [59]:
unlabel_data_q[0]['instances'].pseudo_boxes

Boxes(tensor([[ 657.4378,  299.9938,  760.4671,  359.7356],
        [ 765.3511,  269.6281, 1029.1239,  405.7620],
        [ 621.5120,  301.7120,  683.0803,  345.3318],
        [  17.6943,  287.6151,  113.2596,  337.7801],
        [ 498.5867,  296.2545,  530.5364,  319.8307],
        [ 608.2769,  306.8873,  643.7432,  338.5808],
        [ 569.6702,  301.0650,  601.6965,  328.6351],
        [ 735.0723,  310.4042,  828.6488,  379.3593],
        [ 202.1344,  288.5207,  295.6842,  333.0471],
        [   3.5361,  282.9872,   52.3831,  341.2922],
        [ 169.4056,  282.2881,  237.7195,  321.9904],
        [ 585.1887,  306.8611,  622.1169,  333.6696],
        [1002.3305,  347.5714, 1054.6060,  429.4755],
        [ 371.9883,  294.8872,  403.8795,  312.0861],
        [ 333.9984,  297.4119,  356.9525,  311.9879],
        [ 177.5817,  299.2591,  246.0390,  386.8783],
        [ 274.4435,  297.6523,  308.4064,  317.2697],
        [ 300.6526,  297.0209,  327.9620,  313.8780],
        [ 801.5817,  2

In [5]:
unlabel_data_q = Trainer.resize(unlabel_data_q)    

In [22]:
len(unlabel_data_q)

16

In [29]:
for i in range(16):
    print(unlabel_data_q[i]['instances'].has('gt_boxes'))

False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False


In [35]:
unlabel_data_q[5]

{'file_name': 'data/VOC2007_bddval/JPEGImages/b1c81faa-c80764c5.jpg',
 'image_id': 'b1c81faa-c80764c5',
 'height': 720,
 'width': 1280,
 'instances': Instances(num_instances=11, image_height=600, image_width=1067, fields=[gt_boxes: Boxes(tensor([[659.3727, 228.3333, 672.7101, 240.0000],
         [628.5297, 226.6667, 649.3695, 240.8333],
         [610.1906, 230.0000, 621.8610, 240.8333],
         [587.6836, 231.6667, 597.6867, 240.0000],
         [569.3445, 233.3333, 585.1828, 240.8333],
         [459.3102, 235.0000, 471.8141, 244.1667],
         [470.1469, 235.0000, 482.6508, 245.8333],
         [486.8188, 232.5000, 506.8250, 244.1667],
         [522.6633, 235.0000, 541.0023, 247.5000],
         [527.6649, 235.0000, 556.8406, 256.6667],
         [692.7164, 220.8333, 769.4070, 272.5000]])), gt_classes: tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])]),
 'image': tensor([[[2, 2, 2,  ..., 1, 1, 1],
          [2, 2, 2,  ..., 1, 1, 1],
          [2, 2, 2,  ..., 1, 1, 1],
          ...,
          

In [10]:
( _, proposals_rpn_unsup_k, proposals_roih_unsup_k, _,) = Trainer.model_teacher_list[0](unlabel_data_k, branch="unsup_data_weak")

In [11]:
( _, proposals_rpn_unsup_k2, proposals_roih_unsup_k2, _,) = Trainer.model_teacher_list[1](unlabel_data_k, branch="unsup_data_weak")

In [41]:
len(proposals_roih_unsup_k)

16

In [12]:

def get_match_array_nogt(proposals_roih):
    source_num = len(proposals_roih)
    batch_size = len(proposals_roih[0])
    
    batch_match_array= []
    for data_idx in range(batch_size):

        #pairwise_src2others = []
        match_array_source = []
        for i, source_prediction_n in enumerate(proposals_roih):
            match_array_source_n =[]
            # others
            #pairwise_sa_sb = []        
            for j in range(source_num):
                if j!=i:

                    sourcen_n_match_other = structures.pairwise_iou(source_prediction_n[data_idx].get('pred_boxes'),proposals_roih[j][data_idx].get('pred_boxes'))
                    #pairwise_sa_sb.append(sourcen_n_match_other)
                    match_array_source_n.append(get_match_array(sourcen_n_match_other))
            match_array_source.append(match_array_source_n)  
        batch_match_array.append(match_array_source)
                                                                   
    return  batch_match_array

In [13]:
multi_proposals_roih = [proposals_roih_unsup_k,proposals_roih_unsup_k2]

In [14]:
batch_mt_src = get_match_array_nogt(multi_proposals_roih)

In [15]:
len(batch_mt_src)

16

In [16]:
def bb_ensemble(mt_src,src_idx):
    source_num = len(mt_src[0])
    df_src = pd.DataFrame()    
    src_array = np.array(mt_src[src_idx]).T
    df_src = pd.DataFrame(src_array)
    df_src['summary'] = df_src.sum(axis=1)
    keep_index = df_src.index[df_src.summary==source_num]
    return keep_index

In [17]:
src_idx = 0
batch_size=16
pesudo_proposals_roih_combined = []
for batch_idx in range(batch_size):
    keep_index = bb_ensemble(batch_mt_src[batch_idx],src_idx)
    pesudo_proposals_roih_combined.append( multi_proposals_roih[src_idx][batch_idx][keep_index])

In [19]:
from pt.structures.instances import FreeInstances

def process_pseudo_label2(proposals_roih):
    list_instances = []
    # change to 1 dim temporary
    for proposal_bbox_inst in proposals_roih:

        image_shape = proposal_bbox_inst.image_size
        new_proposal_inst = FreeInstances(image_shape)

        new_bbox_loc = proposal_bbox_inst.pred_boxes.tensor
        pseudo_boxes = Boxes(new_bbox_loc)

                # add boxes to instances
        new_proposal_inst.pseudo_boxes = pseudo_boxes
        new_proposal_inst.scores_logists = proposal_bbox_inst.scores_logists
        if proposal_bbox_inst.has('boxes_sigma'):
            new_proposal_inst.boxes_sigma = proposal_bbox_inst.boxes_sigma
        list_instances.append(new_proposal_inst)
    return list_instances

In [20]:
list_instances = process_pseudo_label2(pesudo_proposals_roih_combined)

In [28]:
roih_combined

Instances(num_instances=8, image_height=600, image_width=1067, fields=[pred_boxes: Boxes(tensor([[694.5236, 220.3513, 776.3004, 275.6724],
        [528.8289, 234.4392, 560.0524, 257.0865],
        [572.5876, 135.3076, 651.8466, 188.6729],
        [499.4250, 131.8156, 587.2648, 192.0158],
        [593.0430, 214.1681, 661.5519, 243.5728],
        [567.9817, 223.9365, 606.1909, 247.8848],
        [460.5973, 231.8435, 491.3189, 250.8547],
        [621.9167, 215.0542, 672.1984, 242.8776]], device='cuda:0',
       grad_fn=<IndexBackward>)), scores: tensor([0.9704, 0.9069, 0.8632, 0.8576, 0.0203, 0.0184, 0.0147, 0.0101],
       device='cuda:0', grad_fn=<IndexBackward>), pred_classes: tensor([0, 0, 0, 0, 0, 0, 0, 0], device='cuda:0'), scores_logists: tensor([[ 3.5231, -3.5369],
        [ 2.7387, -2.7303],
        [ 1.6161, -1.6111],
        [ 2.0313, -1.9718],
        [-1.2048,  1.2886],
        [-1.3112,  1.3979],
        [-1.4033,  1.4871],
        [-1.3437,  1.4549]], device='cuda:0', grad_

In [29]:
from pt.structures.instances import FreeInstances
list_instances = []
#or proposal_bbox_inst in roih_combined:
proposal_bbox_inst =  roih_combined
image_shape = proposal_bbox_inst.image_size
new_proposal_inst = FreeInstances(image_shape)

new_bbox_loc = proposal_bbox_inst.pred_boxes.tensor
pseudo_boxes = Boxes(new_bbox_loc)

        # add boxes to instances
new_proposal_inst.pseudo_boxes = pseudo_boxes
new_proposal_inst.scores_logists = proposal_bbox_inst.scores_logists
if proposal_bbox_inst.has('boxes_sigma'):
    new_proposal_inst.boxes_sigma = proposal_bbox_inst.boxes_sigma
list_instances.append(new_proposal_inst)

In [23]:
list_instances

[FreeInstances(num_instances=8, image_height=600, image_width=1067, fields=[pseudo_boxes: Boxes(tensor([[694.5236, 220.3513, 776.3004, 275.6724],
         [528.8289, 234.4392, 560.0524, 257.0865],
         [572.5876, 135.3076, 651.8466, 188.6729],
         [499.4250, 131.8156, 587.2648, 192.0158],
         [593.0430, 214.1681, 661.5519, 243.5728],
         [567.9817, 223.9365, 606.1909, 247.8848],
         [460.5973, 231.8435, 491.3189, 250.8547],
         [621.9167, 215.0542, 672.1984, 242.8776]], device='cuda:0',
        grad_fn=<IndexBackward>)), scores_logists: tensor([[ 3.5231, -3.5369],
         [ 2.7387, -2.7303],
         [ 1.6161, -1.6111],
         [ 2.0313, -1.9718],
         [-1.2048,  1.2886],
         [-1.3112,  1.3979],
         [-1.4033,  1.4871],
         [-1.3437,  1.4549]], device='cuda:0', grad_fn=<IndexBackward>), boxes_sigma: tensor([[-3.7711, -3.7226, -3.2667, -3.4059],
         [-2.7266, -2.3085, -2.1975, -2.1389],
         [-2.4679, -2.4854, -1.8964, -1.9502],


In [ ]:
process_pseudo_label(
            self, proposals_rpn_unsup_k, proposal_type, psedo_label_method=""
    ):
        list_instances = []
        num_proposal_output = 0.0
        for proposal_bbox_inst in proposals_rpn_unsup_k:
            # all
            if psedo_label_method == "all":
                proposal_bbox_inst = self.threshold_bbox(
                    proposal_bbox_inst, proposal_type=proposal_type
                )
            else:
                raise ValueError("Unkown pseudo label boxes methods")
            num_proposal_output += len(proposal_bbox_inst)
            list_instances.append(proposal_bbox_inst)
        num_proposal_output = num_proposal_output / len(proposals_rpn_unsup_k)
        return list_instances, num_proposal_output

In [ ]:
input proposal_bbox_inst
            image_shape = proposal_bbox_inst.image_size
            new_proposal_inst = FreeInstances(image_shape)

            # create box
            # new_bbox_loc = proposal_bbox_inst.pred_boxes.tensor
            # new_boxes = Boxes(new_bbox_loc)

            # add boxes to instances
            # new_proposal_inst.gt_boxes = new_boxes
            # new_proposal_inst.gt_classes = proposal_bbox_inst.pred_classes
            # new_proposal_inst.scores = proposal_bbox_inst.scores

            # ------------ <all -----------
            new_bbox_loc = proposal_bbox_inst.pred_boxes.tensor
            pseudo_boxes = Boxes(new_bbox_loc)

            # add boxes to instances
            new_proposal_inst.pseudo_boxes = pseudo_boxes
            new_proposal_inst.scores_logists = proposal_bbox_inst.scores_logists
            if proposal_bbox_inst.has('boxes_sigma'):
                new_proposal_inst.boxes_sigma = proposal_bbox_inst.boxes_sigma

        return new_proposal_inst


In [26]:
proposals_roih_unsup_k2[0]

FreeInstances(num_instances=43, image_height=600, image_width=1067, fields=[pred_boxes: Boxes(tensor([[3.1803e+02, 3.0278e+02, 4.0796e+02, 3.5781e+02],
        [4.1591e+01, 2.8276e+02, 2.9419e+02, 4.0292e+02],
        [9.6065e+02, 2.9093e+02, 1.0332e+03, 3.3434e+02],
        [3.8601e+02, 3.0232e+02, 4.4282e+02, 3.4395e+02],
        [3.8055e-02, 3.5004e+02, 6.5678e+01, 4.3995e+02],
        [2.6332e+02, 3.1382e+02, 3.3319e+02, 3.7734e+02],
        [5.3871e+02, 2.9721e+02, 5.6825e+02, 3.2123e+02],
        [7.7827e+02, 2.9077e+02, 8.6378e+02, 3.3127e+02],
        [8.4443e+02, 2.8528e+02, 8.9837e+02, 3.1929e+02],
        [4.2582e+02, 3.0485e+02, 4.5922e+02, 3.3758e+02],
        [4.5356e+02, 3.0418e+02, 4.8237e+02, 3.3065e+02],
        [4.6781e+02, 2.9858e+02, 4.9838e+02, 3.2622e+02],
        [7.5422e+02, 2.9465e+02, 7.8948e+02, 3.1545e+02],
        [4.4647e+02, 3.0575e+02, 4.7286e+02, 3.3301e+02],
        [8.2748e+02, 3.1024e+02, 8.9089e+02, 3.7915e+02],
        [7.2724e+02, 2.9507e+02, 7.5

In [24]:
proposals_roih_unsup_k[0]

FreeInstances(num_instances=60, image_height=600, image_width=1067, fields=[pred_boxes: Boxes(tensor([[ 305.9895,  299.1415,  412.1760,  361.5930],
        [ 382.3076,  300.8235,  446.7187,  345.1768],
        [  35.6888,  270.6293,  310.1724,  405.2239],
        [ 960.8623,  289.7416, 1049.4180,  334.9900],
        [ 538.6613,  294.8270,  573.3475,  320.8658],
        [ 247.0082,  311.1803,  335.0538,  380.8680],
        [ 419.8506,  305.1220,  461.5975,  338.2710],
        [ 463.0506,  299.4230,  500.0391,  328.9950],
        [ 775.6105,  289.7632,  868.5215,  332.7422],
        [   3.6932,  334.4381,   69.2010,  445.4673],
        [ 834.1901,  284.1828,  901.1633,  322.0711],
        [ 446.0519,  305.9645,  483.6690,  332.4266],
        [ 823.1245,  301.1689,  893.0054,  380.1613],
        [ 209.9434,  207.4693,  269.9500,  256.6091],
        [ 560.9183,  286.0779,  587.5471,  311.5070],
        [1065.3970,   15.1283, 1066.7642,  103.0660],
        [1065.5072,    0.0000, 1065.8800, 